<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S105ocrCorrectionV06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# correction of ocr-generated text
- using correction dictionaries
- using rules


In [ ]:
import re, os, sys


In [ ]:
# non-critical, to skip...
!wget https://heibox.uni-heidelberg.de/f/d601ceb0af5a4671a8e7/?dl=1
!mv index.html?dl=1 Parfum_Arm_ABBY.txt

!wget https://heibox.uni-heidelberg.de/f/743a1a57a37c42d8b585/?dl=1
!mv index.html?dl=1 Parfum_Armenian_uncorrected.txt



In [ ]:
# non-critical, to skip...
# dealing with Armenian OCR output with line breaks (is it correct?)

FName = 'Parfum_Armenian_uncorrected.txt'
FNameOut = 'Parfum_Armenian.txt'

FIn = open(FName, 'r')
FOut = open(FNameOut, 'w')

for SLine in FIn:
    SLine = SLine.strip()
    if SLine == '':
        FOut.write('\n\n')
        continue
    if SLine[-1] == '-':
        SLine2write = SLine[:-1]
        FOut.write(SLine2write)
        continue

    FOut.write(SLine + ' ')
FOut.flush()



In [ ]:
# non-critical, to skip...
!wc Parfum_Arm_ABBY.txt
!wc Parfum_Armenian_uncorrected.txt
!wc Parfum_Armenian.txt

In [ ]:
# core starts here - critical

!wget https://heibox.uni-heidelberg.de/f/c977e87cf2b244e6801b/?dl=1
!mv index.html?dl=1 KorpusARM.tgz


In [ ]:
!tar xvzf KorpusARM.tgz
!mkdir KorpusARM1
!mkdir KorpusARM1/stage01
# concatenating files
!cat korpusARM/hyFiktion/* >KorpusARM1/stage01/hyFiktion.txt
!cat korpusARM/hyNatur/* >KorpusARM1/stage01/hyNatur.txt
!cat korpusARM/hyRecht/* >KorpusARM1/stage01/hyRecht.txt
!mkdir KorpusARM1/stage02

# function for Armenian line breaks:

def correctLineBreaksHY(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')
    countHyphens = 0
    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '':
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            countHyphens +=1
            continue
        FOut.write(SLine + ' ')
    FOut.flush()
    print(str(countHyphens) + ' hyphens corrected')
    return

correctLineBreaksHY('KorpusARM1/stage01/hyFiktion.txt', 'KorpusARM1/stage02/hyFiktion.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyNatur.txt', 'KorpusARM1/stage02/hyNatur.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyRecht.txt', 'KorpusARM1/stage02/hyRecht.txt')


In [ ]:

!wc KorpusARM1/stage02/hyFiktion.txt
!wc KorpusARM1/stage02/hyNatur.txt
!wc KorpusARM1/stage02/hyRecht.txt

In [ ]:
# Dowloading the file with corrections
# !wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
# without ճեր
!wget https://heibox.uni-heidelberg.de/f/4a24540473564788853d/?dl=1

!mv index.html?dl=1 Pilot-Corrections-all.tsv
!wc Pilot-Corrections-all.tsv



In [ ]:
# downloading wikipedia
### downloading Armenian Wikipedia
!wget https://heibox.uni-heidelberg.de/f/d1f866a61bd545318213/?dl=1
!mv index.html?dl=1 hywiki-20221101-pages-articles.txt.gz
!gunzip hywiki-20221101-pages-articles.txt.gz


--2023-07-30 13:37:56--  https://heibox.uni-heidelberg.de/f/d1f866a61bd545318213/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/0a5ddf37-ea57-446d-835f-52f40e5c07aa/hywiki-20221101-pages-articles.txt.gz [following]
--2023-07-30 13:37:57--  https://heibox.uni-heidelberg.de/seafhttp/files/0a5ddf37-ea57-446d-835f-52f40e5c07aa/hywiki-20221101-pages-articles.txt.gz
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 193021637 (184M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 184.08M  14.9MB/s    in 12s     

2023-07-30 13:38:09 (15.2 MB/s) - ‘index.html?dl=1’ saved [193021637/193021637]



In [ ]:
# the length of wikipedia
!wc hywiki-20221101-pages-articles.txt

  2446411  56531792 803098410 hywiki-20221101-pages-articles.txt


In [ ]:
# non-critical
def readCorrections(colNumberOri, colNumberCorrect, SFIn, SFOut = None):
    LTWrongCorrect = []
    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.strip()
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\n')
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect

# critical path function
def readCorrectionsFrq(colNumberOri, colNumberCorrect, colNumberFrq, SFIn, SFOut = None):
    LTWrongCorrect = []
    '''
    if type(LTWrongCorrect) == list:
        pass
    '''

    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.rstrip('\n')
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            SFrq = LLine[colNumberFrq]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]', f'{SFrq}')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect, SFrq in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\t{SFrq}\n')
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect


In [ ]:
# reading corrections for word forms, with the purpose of generalizing them
# goal: to display candidates for correction -- based on existing corrections (?)
LTWrongCorrectWordF, DWrongCorrectWordF = readCorrectionsFrq(1, 4, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
# LTWrongCorrectLemmaF, DWrongCorrectLemmaF = readCorrectionsFrq(3, 6, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')
print(LTWrongCorrectWordF)
# print(LTWrongCorrectLemmaF)
# ինձ|ինչ
# առջև|առջևից
# ինչ|ինձ
# ինչ|ես
# գիտենալ|իմանալ


ինճ	ինձ	ինչ
առջնից	առջև	առջևից
ինճ	ինչ	ինձ
171
[('[մերճակա]', '[մերձակա]', '4'), ('[առջն]', '[առջև]', '4'), ('[թեթնություն]', '[թեթևություն]', '4'), ('[Եթենա]', '[եթե նա]', '4'), ('[ննա]', '[նա]', '4'), ('[ճեռքերն]', '[ձեռքից]', '4'), ('[ճայն]', '[ձայն]', '4'), ('[ճեռքով]', '[ձեռքով]', '4'), ('[այլնս]', '[այլևս]', '4'), ('[ետնի]', '[ետևի]', '4'), ('[կեղնները]', '[կեղևները]', '4'), ('[ճկան]', '[ձկան]', '4'), ('[ննա]', '[նա]', '4'), ('[բանաձնի]', '[բանաձևի]', '4'), ('[առնտրական]', '[առևտրական]', '4'), ('[արնելյան]', '[արևելյան]', '4'), ('[կուղնորվի]', '[կուղևորվի]', '4'), ('[նս]', '[ևս]', '4'), ('[հետնեց]', '[հետևել]', '4'), ('[նուխիսկ]', '[նույնիսկ]', '4'), ('[երնույթ]', '[երևույթ]', '4'), ('[քրտնքով]', '[քրտինքով]', '4'), ('[արվարճանում]', '[արվարձանում]', '4'), ('[անճամբ]', '[անձամբ]', '4'), ('[ճայնը]', '[ձայնը]', '4'), ('[ճգվում]', '[ձգվում]', '4'), ('[ճիու]', '[ձիու]', '4'), ('[դարճնում]', '[դարձնում]', '4'), ('[ուղնորվում]', '[ուղևորվում ]', '4'), ('[իջնանատան]', '[իջևանատան]', '4'

In [ ]:
print(len(DWrongCorrectWordF))
# print(len(DWrongCorrectLemmaF))


171


In [ ]:
for key, value in sorted(DWrongCorrectWordF.items()):
    print(f'{key}\t{value}')

In [ ]:
'''
for i in range(5):
    LKeys = list(DWrongCorrectLemmaF.keys())
    SKey = LKeys[i]
    SVal = DWrongCorrectWordF[SKey]
    print(f'{SKey} {SVal}\n')
'''


This section applies corrections to an Armenian texts and records which words have been corrected

In [ ]:
# critical path function...
# function for Armenian tokenization
def tokenizeTextHY(SFIn):
    LLParagraphs = []
    with open(SFIn, 'r') as FIn:
        countpara = 0
        for SLine in FIn:
            countpara += 1
            if countpara % 100000 == 0: print(countpara)
            SLine = SLine.strip()
            if SLine == '': continue
            LLine = re.split('([ ,\.\:։;\'\"\(\)\-\–\!\?\{\}\t\«\»]+)', SLine)
            # if LLine == '': continue
            if LLine: LLParagraphs.append(LLine)
    return LLParagraphs


# not on critical path ?... skip...?
# def printLLParagraphs(SFIn, SFOut, DCorrections1 = None, DCorrections2 = None):
def printLLParagraphs(SFIn, SFOut, DCorrections1 = None):
    DErrors = {}
    LLParagraphs = tokenizeTextHY(SFIn)
    LCorrected = []
    FOut = open(SFOut, 'w')
    for LPara in LLParagraphs:
        # if LPara == [] or LPara == ['']: continue
        if LPara == []: continue
        # FOut.write(str(LPara) + '\n')
        LParaNew = []
        # FOut.write('<p>\n')

        # for each word
        i = 0 # counting words
        for el in LPara:
            i+=1 # index of next word
            # print(el, i)
            # if el == ' ': continue
            # if there is no correction dictionary, then just copy the string -- if dictionary is not provided
            if not DCorrections1:
                elCorrect = el
                continue

            if el in DCorrections1:
                iStart = i-5
                if iStart <0: iStart = 0
                iFinish = i+5
                # if iFinish >len(LPara): iFinish = len(LPara)
                LContext = LPara[iStart : iFinish]
                elCorrect = DCorrections1[el]
                LCorrected.append((el, elCorrect, LContext, LPara)) # word context for the corrected word

            else:
                elCorrect = el

            LParaNew.append(elCorrect)




        SParaNew = ' '.join(LParaNew)
        FOut.write(f'{SParaNew}\n')

        # FOut.write('</p>\n')
    FOut.flush()
    return LCorrected, DErrors

In [ ]:
# not on critical path ? ... skip ... ?
def printCorrections(LCorrected, SFOut):
    FOut1 = open(SFOut, 'w')
    for (el, elCorr, LContext, LPara) in sorted(LCorrected):
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)
        FOut1.write( f'{el} \t {elCorr} \n\tcontext: {SContext} \n\tparagraph: {SPara}\n\n' )
    return

In [ ]:
!head --lines=40 /content/Parfum_Armenian.txt

In [ ]:
# not on critical path ?
LCorrected1, DErrors1 = printLLParagraphs('/content/Parfum_Arm_ABBY.txt', '/content/Parfum_Arm_ABBY-corrected.txt', DWrongCorrectWordF)
printCorrections(LCorrected1, '/content/Parfum_Arm_ABBY-changes.txt')
print(len(LCorrected1))
LCorrected2, DErrors2 = printLLParagraphs('/content/Parfum_Armenian.txt', '/content/Parfum_Armenian-corrected.txt', DWrongCorrectWordF)
printCorrections(LCorrected2, '/content/Parfum_Armenian-changes.txt')
print(len(LCorrected2))


6
913


In [ ]:

'''
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/Parfum_Arm_ABBY-changes.txt', 'w')
    for (el, elCorr, LContext, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)
        FOut1.write( f'{el}\t{elCorr}\t{SContext}\t{SPara}\n\n' )

print(len(LCorrected2))
if LCorrected2:
    FOut2 = open('/content/Parfum_Armenian-changes.txt', 'w')
    for (el, elCorr, LContext, LPara) in LCorrected2:
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)
        FOut2.write( f'{el}\t{elCorr}\t{SContext}\t{SPara}\n\n' )
'''

'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyFiktion-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''

'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyNatur-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''


'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyRecht-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''


In [ ]:
# not on critical path?
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyFiktion.txt', '/content/KorpusARM1/stage02/hyFiktion-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1, DErrors1 = printLLParagraphs('/content/KorpusARM1/stage02/hyFiktion.txt', '/content/KorpusARM1/stage02/hyFiktion-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyFiktion-changes.txt')



1118


In [ ]:
# not on critical path?
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyNatur.txt', '/content/KorpusARM1/stage02/hyNatur-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1, DErrors1 = printLLParagraphs('/content/KorpusARM1/stage02/hyNatur.txt', '/content/KorpusARM1/stage02/hyNatur-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyNatur-changes.txt')



586


In [ ]:
# not on critical path?
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyRecht.txt', '/content/KorpusARM1/stage02/hyRecht-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1, DErrors1 = printLLParagraphs('/content/KorpusARM1/stage02/hyRecht.txt', '/content/KorpusARM1/stage02/hyRecht-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyRecht-changes.txt')



733


## Critical path section
### Wikipedia

In [ ]:
try:
    del LLParaWiki
except:
    pass

In [ ]:
LLParaWiki = tokenizeTextHY('/content/hywiki-20221101-pages-articles.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000


In [ ]:
print(LLParaWiki[1])

['Հայաստան', ' , ', 'պաշտոնական', ' ', 'անվանումը՝', ' ', 'Հայաստանի', ' ', 'Հանրապետություն', ', ', 'պետություն', ' ', 'Առաջավոր', ' ', 'Ասիայում՝', ' ', 'Հայկական', ' ', 'լեռնաշխարհի', ' ', 'հյուսիսարևելյան', ' ', 'մասում', '։ ', 'Քաղաքական', ' ', 'և', ' ', 'մշակութային', ' ', 'իմաստով', ', ', 'սակայն', ', ', 'գտնվում', ' ', 'է', ' ', 'հարավարևելյան', ' ', 'Եվրոպայի', ' ', 'Կովկասյան', ' ', 'տարածաշրջանում', '։ ', 'Հյուսիսում', ' ', 'սահմանակցում', ' ', 'է', ' ', 'Վրաստանին', ', ', 'արևելքում՝', ' ', 'Ադրբեջանին', ', ', 'հարավում՝', ' ', 'Իրանին', ', ', 'իսկ', ' ', 'արևմուտքում՝', ' ', 'Թուրքիային', '։ ', 'Հարավարևելյան', ' ', 'կողմում', ' ', 'Բերձորի', ' ', 'միջանցքով', ' ', 'կապվում', ' ', 'է', ' ', 'Արցախի', ' ', 'Հանրապետությանը', ', ', 'իսկ', ' ', 'հարավ', '-', 'արևմուտքում', ' ', 'Ադրբեջանի', ' ', 'էքսկլավ', ' ', 'Նախիջևանի', ' ', 'Ինքնավար', ' ', 'Հանրապետությունն', ' ', 'է', '։ ', 'Այժմյան', ' ', 'ՀՀ', '-', 'ն', ' ', 'զբաղեցնում', ' ', 'է', ' ', 'պատմական', ' ', 'Հայաստանի', 

In [ ]:
len(LLParaWiki[1])

137

In [ ]:
len(LLParaWiki)

2153019

In [ ]:
def llPara2dict(LLParagraphs):
    DFreq = {}
    p=0
    for LPara in LLParagraphs:
        p+=1
        if p%200000 == 0:
            print(p)
        # if LPara == [] or LPara == ['']: continue
        if LPara == []: continue
        # FOut.write(str(LPara) + '\n')
        i = 0 # counting words
        for el in LPara:
            i+=1 # index of next word
            try:
                DFreq[el] += 1
            except:
                DFreq[el] = 1
    return DFreq


In [ ]:
DWiki = llPara2dict(LLParaWiki)
print(len(DWiki))

200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2071984


In [ ]:
len(DWiki)

2071984

In [ ]:
FOut = open('hywiki-frqDict.txt', 'w')
for key, val in sorted(DWiki.items(), key=lambda item: item[1], reverse=True):
    FOut.write(f'{key}\t{val}\n')

In [ ]:
!wc hywiki-frqDict.txt

 2071974  4211029 42482907 hywiki-frqDict.txt


In [ ]:
!head --lines=40 hywiki-frqDict.txt

## Discover candidate rewriting rules systematically

In [ ]:
print(len(LTWrongCorrectWordF))

192


In [ ]:
for SWrong, SCorrect, Frq in LTWrongCorrectWordF:
    print(SWrong, SCorrect)

In [ ]:
def getPrefInfSuf(wrd1, wrd2):
    try:
        cpref = os.path.commonprefix([wrd1, wrd2])
        drw1 = wrd1[::-1]
        drw2 = wrd2[::-1]
        ffusc = os.path.commonprefix([drw1, drw2])
        csuff = ffusc[::-1]
    except:
        sys.stderr.write('error finding pref- and suffix')
        cpref = None
        csuff = None

    try:
        wrd1minpref = wrd1.removeprefix(cpref)
        wrd2minpref = wrd2.removeprefix(cpref)
        wrd1centre = wrd1minpref.removesuffix(csuff)
        wrd2centre = wrd2minpref.removesuffix(csuff)
    except:
        sys.stderr.write('error finding centre 1 and 2')
        wrd1centre = None
        wrd2centre = None

    return cpref, wrd1centre, wrd2centre, csuff


P12, I1, I2, S12 = getPrefInfSuf('[перепливи]', '[перелови]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[розгубився]', '[розгубивсь]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[вловив]', '[зловив]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкодити]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)




P12, I1, I2, S12 = getPrefInfSuf('[մերճակա]', '[մերձակա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[առջն]', '[առջև]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[թեթնություն]', '[թեթևություն]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[Եթենա]', '[եթե նա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ննա]', '[նա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ճեռքերն]', '[ձեռքից]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)




[пере  |  пли  >  ло  |  ви]
[розгубивс  |  я  >  ь  |  ]
[  |  в  >  з  |  ловив]
[пере  |  х  >  шк  |  одити]
[пере  |  ходи  >  шкоджа  |  ти]
[  |  ходи  >  перешкоджа  |  ти]
[մեր  |  ճ  >  ձ  |  ակա]
[առջ  |  ն  >  և  |  ]
[թեթ  |  ն  >  և  |  ություն]
[  |  Եթե  >  եթե   |  նա]
[ն  |    >  ա]  |  նա]
[  |  ճեռքերն  >  ձեռքից  |  ]


In [ ]:
'''

# functions to compare two strings
# import os
def getCmnPrefix(S1, S2):
    common = os.path.commonprefix([S1, S2])
    return common


def getCmnSuffix(S1,S2):
    S1r = S1[::-1]
    S2r = S2[::-1]
    commonR = getCmnPrefix(S1r, S2r)
    common = commonR[::-1]
    return common

def getDifInfix(S1, S2, LContext = 0, RContext = 0):
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    diffInfix1 = S1[IPLen-LContext:-1*(ISLen)+RContext]
    # print('S1', S1, IPLen-LContext, -1*(ISLen)+RContext)
    diffInfix2 = S2[IPLen-LContext:-1*(ISLen)+RContext]
    # print('S2', S2, IPLen-LContext, -1*(ISLen)+RContext)
    return(diffInfix1, diffInfix2)

def getDifSuffix(S1, S2, LContext = 0):
    # used if common suffix == ''
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    if ISLen == 0:
        diffSuffix1 = S1[IPLen-LContext:]
        diffSuffix2 = S2[IPLen-LContext:]
    else:
        diffSuffix1 = ''
        diffSuffix2 = ''
    return(diffSuffix1, diffSuffix2)

def getDifPrefix(S1, S2, RContext = 0):
    # used if common suffix == ''
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    if IPLen == 0:
        diffPrefix1 = S1[:-1*(ISLen)+RContext]
        diffPrefix2 = S2[:-1*(ISLen)+RContext]
    else:
        diffPrefix1 = ''
        diffPrefix2 = ''

    return(diffPrefix1, diffPrefix2)

def getDiff(S1, S2, LContext = 0, RContext = 0):
    IPLen = len(getCmnPrefix(S1, S2))
    ISLen = len(getCmnSuffix(S1, S2))
    if IPLen and ISLen:
        diff1 = S1[IPLen-LContext:-1*(ISLen)+RContext]
        diff2 = S2[IPLen-LContext:-1*(ISLen)+RContext]
    elif IPLen: # there is a common prefix, find different suffixes
        diff1 = S1[IPLen-LContext:]
        diff2 = S2[IPLen-LContext:]
    elif ISLen: # there is a common suffix, find different prefixes
        diff1 = S1[:-1*(ISLen)+RContext]
        diff2 = S2[:-1*(ISLen)+RContext]
    else:
        diff1 = S1
        diff2 = S2
    return (diff1, diff2)


def getPrefInfSuf(S1, S2):
    P12 = ''
    I1 = ''
    I2 = ''
    S12 = ''
    try:
        I1, I2 = getDifInfix(S1, S2, LContext = 0, RContext = 0)
    except:
        I1 = None
        I2 = None
    try:
        P12 = getCmnPrefix(S1, S2)
    except:
        P12 = None
    try:
        S12 = getCmnSuffix(S1, S2)
    except:
        S12 = None

    return P12, I1, I2, S12

# testing
P12, I1, I2, S12 = getPrefInfSuf('[перепливи]', '[перелови]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[розгубився]', '[розгубивсь]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[вловив]', '[зловив]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкодити]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)



'''

In [ ]:
'''
# testing
P12 = getCmnPrefix('[перепливи]', '[перелови]')
S12 = getCmnSuffix('[перепливи]', '[перелови]')
# I1, I2 = getDifInfix('перепливи', 'перелови')
# print(P12, S12)
# print(I1, I2)

I1, I2 = getDifInfix('[перепливи]', '[перелови]', LContext = 0, RContext = 0)
print('rule:', I1, I2)
print('prefix-suffix:', P12, S12)

S1, S2 = getDifSuffix('розгубився', 'розгубивсь', LContext = 0)
print(S1, S2)
S1, S2 = getDifSuffix('розгубився', 'розгубивс', LContext = 0)
print(S1, S2)

S1, S2 = getDifPrefix('вловив', 'зловив', RContext = 0)
print(S1, S2)
S1, S2 = getDifPrefix('ловив', 'зловив', RContext = 0)
print(S1, S2)

print('new version....')
I1, I2 = getDiff('перепливи', 'перелови', LContext = 1, RContext = 1)
print(I1, I2)

S1, S2 = getDiff('розгубився', 'розгубивсь', LContext = 1)
print(S1, S2)
S1, S2 = getDiff('розгубився', 'розгубивс', LContext = 1)
print(S1, S2)

S1, S2 = getDiff('вловив', 'зловив', RContext = 1)
print(S1, S2)
S1, S2 = getDiff('ловив', 'зловив', RContext = 1)
print(S1, S2)

'''


In [ ]:
def createSufList(a):
    ''' odyty] --> odyty], odyty, odyt, ody, od, o, "" '''
    LSuf = []
    for j in range(1,len(a)+1):
        prefix = a[:j]
        LSuf.append(prefix)
        # print(j, len(prefix), LSuf)
    LSuf.insert(0, '')
    return LSuf

createSufList('odyty]')

['', 'o', 'od', 'ody', 'odyt', 'odyty', 'odyty]']

In [ ]:
createSufList('ություն]')

['', 'ո', 'ու', 'ութ', 'ությ', 'ությո', 'ությու', 'ություն', 'ություն]']

In [ ]:
def createPrefList(s):
    LPref = [s[-i:] for i in range(1, len(s) + 1)]
    LPref.insert(0, '')
    # print(LPref)
    return LPref

createPrefList('[pere')

['', 'e', 're', 'ere', 'pere', '[pere']

In [ ]:
createPrefList('[թեթ')

['', 'թ', 'եթ', 'թեթ', '[թեթ']

### we create the list of potential rewrite rules


In [ ]:
def twoWords2listOfRules(W1, W2):
    DTRules = {}
    DTRulesLen = {}
    LTRules = []
    P12, I1, I2, S12 = getPrefInfSuf(W1, W2)
    LPref12 = createPrefList(P12)
    LSuf12 = createSufList(S12)

    for pref in LPref12:
        for suf in LSuf12:
            SlhsRule = pref + I1 + suf
            SrhsRule = pref + I2 + suf
            TRules = (SlhsRule, SrhsRule)
            LTRules.append(TRules)
            try:
                DTRules[TRules] += 1
            except:
                DTRules[TRules] = 1

            try:
                DTRulesLen[TRules] = len(SlhsRule)
            except:
                continue
    return LTRules, DTRules, DTRulesLen

def printFrqTDict(DTFrq, FOut = None):
    for key, val in sorted(DTFrq.items(), key=lambda x:x[1], reverse=True):
        LHS, RHS = key
        if FOut:
            FOut.write(f'{LHS}, {RHS}, {str(val)}\n')
        else:
            print(LHS, RHS, str(val))
    if FOut:
        FOut.flush()

def printTList(LTuples, FOut = None):
    for el in LTuples:
        LHS, RHS = el
        if FOut:
            FOut.write(f'{LHS}, {RHS}\n')
        else:
            print(LHS, RHS)
    if FOut:
        FOut.flush()

LTRules, DTRules, DTRulesLen = twoWords2listOfRules('[перепливи]', '[перелови]')
FOut = open('printFrqTDict-test.txt', 'w')
printFrqTDict(DTRules, FOut)
FOut0 = open('printFrqTDictLen-test.txt', 'w')
printFrqTDict(DTRulesLen, FOut0)
FOut1 = open('printTList-test.txt', 'w')
printTList(LTRules, FOut1)

In [ ]:
LTRules, DTRules, DTRulesLen = twoWords2listOfRules('[թեթնություն]', '[թեթևություն]')
FOut0 = open('printTRulesLen-test-hy.txt', 'w')
printFrqTDict(DTRulesLen, FOut0)


In [ ]:
def findLongestMatch(SInput, DTRulesLength):
    SOutput = None
    for key, val in sorted(DTRulesLength.items(), key=lambda x:x[1], reverse=True):
        LHS, RHS = key
        if LHS in SInput:
            SOutput = SInput.replace(LHS, RHS, 1)
            break
    return SOutput

SOutput1 = findLongestMatch('[перепливи]', DTRulesLen)
if SOutput1: print(SOutput1)

SOutput1 = findLongestMatch('[перешкоджав]', DTRulesLen)
if SOutput1: print(SOutput1)

SOutput1 = findLongestMatch('[проходжав]', DTRulesLen)
if SOutput1: print(SOutput1)


[перелови]
